In [0]:

from sklearn.model_selection import train_test_split
import sklearn

#Pipeline
from sklearn.pipeline import Pipeline
from sklearn import set_config
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import Pipeline as PipelineSampling

#Sampling
from imblearn.over_sampling import RandomOverSampler

#Feature enginner
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PowerTransformer

#Classifier
from sklearn.ensemble import RandomForestClassifier
import pickle

In [0]:
!pip install imblearn
from sklearn.pipeline import make_pipeline

     |████████████████████████████████| 199 kB 4.8 MB/s 
     |████                            | 3.3 MB 5.5 MB/s et

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 26.7 MB 20.2 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
!pip install mlflow -q

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
import mlflow
import mlflow.sklearn

In [0]:
#Leemos el dataset
df = spark.sql("select * from default.diabetes_3_csv")
df = df.toPandas()

In [0]:
X = df.drop(columns=['Outcome'])
y = df.Outcome
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [0]:
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
dbutils.fs.put("file:///root/.databrickscfg","[DEFAULT]\nhost=https://community.cloud.databricks.com\ntoken = "+token,overwrite=True)

Wrote 98 bytes.
Out[16]: True

In [0]:
rf = RandomForestClassifier(ccp_alpha=0.0, criterion= 'entropy', n_estimators=55, min_samples_split=0.1, min_samples_leaf=3, max_features=6, max_depth=5.0)

mlflow.autolog(exclusive=False)

with mlflow.start_run():
    var_quantile = ["Pregnancies", "DiabetesPedigreeFunction", "Age"]

    quantile_transformer = Pipeline(
        steps=[("imputer", IterativeImputer(n_nearest_features=4)), 
                ("scaler", QuantileTransformer(n_quantiles=200, output_distribution='normal'))])

    var_transf = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]
    quantile_transformer = Pipeline(
    steps=[("imputer", IterativeImputer(n_nearest_features=4)), 
           ("scaler", PowerTransformer(method='yeo-johnson'))])

    preprocessor = ColumnTransformer( transformers=[
        ("quantile", quantile_transformer, var_quantile),
        ("transf", quantile_transformer, var_transf),])

    model = PipelineSampling(steps=[("preprocessor", preprocessor), 
                             ('resampling',RandomOverSampler()),
                             ("classifier", rf)])

    model = model.fit(X_train, y_train)
    
    y_train_predicted = model.predict(X_train)
    accuracy_train = (model.predict(X_train) == y_train).mean()
    accuracy_test  = (model.predict(X_test)  == y_test).mean()

    mlflow.log_metric("accuracy_train", accuracy_train)
    mlflow.log_metric("accuracy_test", accuracy_test)
    
    mlflow.sklearn.log_model(model, "model")
    
    mlflow.set_tag("owner","Pedro")
    mlflow.set_tag("periodo","Clase 4 ml2")
    

2022/04/14 22:58:57 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2022/04/14 22:58:57 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during spark autologging: Exception while attempting to initialize JVM-side state for Spark datasource autologging. Please create a new Spark session and ensure you have the mlflow-spark JAR attached to your Spark session as described in http://mlflow.org/docs/latest/tracking.html#automatic-logging-from-spark-experimental. Exception:
'JavaPackage' object is not callable
2022/04/14 22:58:57 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.ml.
2022/04/14 22:58:57 WARNING mlflow.sklearn: Failed to infer model signature: the trained model does not specify a `predict` function, which is required in order to infer the signature
2022/04/14 22:59:00 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp3fsw03qe/model/model.p

In [0]:
model_name = "uba_ml2"

result = mlflow.register_model("runs:/14073543e66b472796c2f89ed1c68e0c/model",model_name)
# result = mlflow.register_model("runs:/5683bbfa87d44e52a16d8ac23e1b2573/model",model_name)

In [0]:
model_name = "uba_ml2"

result = mlflow.register_model('runs:/8d6baa251cc14ccda78c113ce9452dae/model',model_name)


---------------------------------------------------------------------------
RestException                             Traceback (most recent call last)
<command-1081509836847049> in <module>
      1 model_name = "uba_ml2"
      2 
----> 3 result = mlflow.register_model('runs:/8d6baa251cc14ccda78c113ce9452dae/model',model_name)

/databricks/python/lib/python3.8/site-packages/mlflow/tracking/_model_registry/fluent.py in register_model(model_uri, name, await_registration_for)
     65             )
     66         else:
---> 67             raise e
     68 
     69     if RunsArtifactRepository.is_runs_uri(model_uri):

/databricks/python/lib/python3.8/site-packages/mlflow/tracking/_model_registry/fluent.py in register_model(model_uri, name, await_registration_for)
     56     client = MlflowClient()
     57     try:
---> 58         create_model_response = client.create_registered_model(name)
     59         eprint("Successfully registered model '%s'." % create_model_response.name)
     60  

In [0]:
pickle.dump(model, open('model.pkl', 'wb'))